In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

import seaborn as sns

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 10

%matplotlib inline

## 일별 데이터의 경우 

In [ ]:
# 마이너스 포함 데이터 사용
df = pd.read_excel('C:/Users/Sum/Desktop/Sumin/16.프로젝트/한경_농협양곡/00.데이터/시계열용.xlsx',sheet_name='price2')
df.head()

In [ ]:
df['날짜'] = pd.to_datetime(df['날짜'])
df.info()

In [ ]:
# 그래프 살펴보기
plt.figure(figsize=(25,10))
plt.plot(df.날짜, df.공급금액)
plt.title("거래 날짜에 따른 공급금액 그래프")
plt.xlabel('날짜')
plt.ylabel('공급금액')
plt.show()

데이터의 등락폭이 너무나 커서 이런 현상이 발생하는 것 같음.  
동시에 연도별로 살펴보면, 2019년을 제외하고는 하반기에 가서 매출이 증가하고 있다.  
그러나 점진적으로 증가하고 있는 추세를 보이기 때문에 1차 차분으로 해결해보려고 함.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df.공급금액, model='additive',freq=7)

fig = plt.figure()
fig = result.plot()
fig.set_size_inches(20,15)

In [ ]:
# ACF
'''import statsmodels.api as sm
fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df.공급금액,lags=20,ax=ax1)'''


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(df.공급금액)
plot_pacf(df.공급금액)
plt.show()

In [ ]:

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(yymm.공급금액)
plot_pacf(yymm.공급금액)
plt.show()

In [35]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(df.공급금액)
print(f'p-values : {result[1]}')

p-values : 0.03226256686006423


In [ ]:
# 1차 차분
df_dff = df - df.shift()
df_dff.head()

In [ ]:
plt.figure(figsize=(25,10))
plt.plot( df_dff.공급금액)
plt.title("거래 날짜에 따른 공급금액 그래프")
plt.xlabel('날짜')
plt.ylabel('공급금액')
plt.show()


1차 차분 후 나아졌다고 이게 볼수가 있는건지....

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
order=(1,1,1) # AR, 차분, MA
model = ARIMA(df.공급금액,order)
rfit = model.fit()
rfit.summary()

## 월별 데이터의 경우

In [ ]:
yymm = pd.read_excel('C:/Users/Sum/Desktop/Sumin/16.프로젝트/한경_농협양곡/00.데이터/시계열용.xlsx',sheet_name='yearmonth')
# 2021년 7월 데이터의 경우 혼자 너무 튀어서 삭제
yymm.head()

In [ ]:
yymm.날짜 = pd.to_datetime(yymm.날짜)
del yymm['매출수량']
yymm.info()

In [ ]:
# 그래프 살펴보기
plt.figure(figsize=(25,10))
plt.plot(yymm.날짜,yymm.공급금액)
plt.title("월별 거래 날짜에 따른 공급금액 그래프")
plt.xlabel('날짜')
plt.ylabel('공급금액')
plt.show()

In [ ]:
# 0이 존재하므로 가법으로 진행, 월이므로 freq는 12
result = seasonal_decompose(yymm.공급금액, model='additive',freq=12)

fig = plt.figure()
fig = result.plot()
fig.set_size_inches(20,15)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(yymm.공급금액)
plot_pacf(yymm.공급금액)
plt.show()

ACF를 살펴보면 자기상관이 있는 것으로 판단됨.  
절단값 추정 : 4  
PACF에서는 2
